In [ ]:
from google.colab import drive
import os
import sys
if 'google.colab' in str(get_ipython()):
  if 'drive' not in os.listdir():
      drive.mount('/content/drive')
sys.path.insert(0,'/content/drive/MyDrive/pytorch-lifestream/seq_encoders')

# Скрипт по обучению coles модели

## Импорт модулей и подгрузка конфига

In [ ]:
# !pip install -q pytorch-lifestream==0.5.2
# !pip install -q pytorch-lightning==1.6.*
# !pip install -q catboost
# !pip install -q s5-pytorch

In [ ]:
from functools import partial
import os
import yaml
import joblib
import gc

from catboost import CatBoostClassifier, metrics

import torchmetrics
from sklearn.model_selection import train_test_split

import numpy as np
import pandas as pd

import torch
import warnings
from torch import nn
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from ptls.nn import TrxEncoder, RnnSeqEncoder,TransformerSeqEncoder,TransformerEncoder
from ptls.frames.coles import CoLESModule
from ptls.data_load.datasets import MemoryMapDataset
from ptls.data_load.iterable_processing import ISeqLenLimit,FeatureFilter,SeqLenFilter
from ptls.data_load.utils import collate_feature_dict
from ptls.frames.coles import ColesDataset
from ptls.frames.coles.split_strategy import SampleSlices,SampleUniform
from ptls.frames import PtlsDataModule
from ptls.frames.inference_module import InferenceModule
from ptls.data_load.utils import collate_feature_dict
from ptls.data_load.datasets import inference_data_loader
import ptls
# import own models
from simple_seq_encoder import SimpleSeqEncoder
from new_rnn_encoder import NewRnnEncoder

import logging
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# sberhack_gender_prediction_data, rosbank_churn_prediction_data, datafusion_churn_prediction_data
# ['gru','lstm','cornn','urnn','indrnn','qrnn','lmu','lem','lru','s5']

# lru мимо - без jax видимо не работает
# очень долго: 'cornn','urnn','lmu', 'lru' на больших датасетах
# для sberhack_gender_prediction_data / datafusion_churn_prediction_data пробуем indrnn,qrnnn,lem,s5

data_type = 'datafusion_churn_prediction_data'
model_type = '128'
list(filter(lambda x: model_type in x and data_type.replace('_data','') in x and 'coles' in x,
            os.listdir('drive/My Drive/pytorch-lifestream/configs')))

['datafusion_churn_predictioncoles_gru_128_1.yaml',
 'datafusion_churn_predictioncoles_lstm_128_1.yaml',
 'datafusion_churn_predictioncoles_cornn_128_1.yaml',
 'datafusion_churn_predictioncoles_urnn_128_1.yaml',
 'datafusion_churn_predictioncoles_indrnn_128_1.yaml',
 'datafusion_churn_predictioncoles_qrnn_128_1_1.yaml',
 'datafusion_churn_predictioncoles_lmu_128_1.yaml',
 'datafusion_churn_predictioncoles_lem_128_1.yaml',
 'datafusion_churn_predictioncoles_lru_128_1.yaml',
 'datafusion_churn_predictioncoles_s5_128_1.yaml',
 'datafusion_churn_predictioncoles_rmt_128.yaml',
 'datafusion_churn_predictioncoles_transformer_128.yaml']

In [ ]:
config_name = 'datafusion_churn_predictioncoles_s5_128_1.yaml'
path_to_working_directory = 'drive/My Drive/pytorch-lifestream'

with open(os.path.join(path_to_working_directory,'configs',config_name),'r') as f:
  model_config = yaml.safe_load(f)

with open(os.path.join(path_to_working_directory,'configs',model_config['data_config']),'r') as f:
  data_config = yaml.safe_load(f)

In [ ]:
# опредедяем i filters и splitter
i_filters = [SeqLenFilter(min_seq_len=model_config['min_seq_len']),FeatureFilter(drop_feature_names=[data_config['target_col']])]
splitter = SampleSlices(**model_config['splitter_params'])

##Загрузка предобработанных данных

In [ ]:
df_data_train = joblib.load(os.path.join(data_config['path_folder'],'train_'+model_config['data_config'].replace('yaml','pickle')))
df_data_valid = joblib.load(os.path.join(data_config['path_folder'],'valid_'+model_config['data_config'].replace('yaml','pickle')))
# df_data_test = joblib.load(os.path.join(data_config['path_folder'],'test_'+model_config['data_config'].replace('yaml','pickle')))
preprocessor = joblib.load(os.path.join(data_config['path_folder'],'preprocessor_'+model_config['data_config'].replace('yaml','pickle')))

## Построение моделей

In [ ]:
# если надо поменять, то сами ручками в файле все меняем
if data_config.get('trx_embed_dim',False) == False:
    cat_feature_params = {k: {'in' : v, 'out' : v // model_config['rnn_config']['category_emb_dim_reduction']} for k,v in preprocessor.get_category_dictionary_sizes().items()}
else:
    cat_feature_params = data_config['trx_embed_dim']

if sum([v['out'] for k,v in cat_feature_params.items()]) % 2 == 1:
    cat_feature_params[list(cat_feature_params.keys())[0]]['out'] += 1

num_feature_params = {f:'identity' for f in data_config['numeric_cols']}
trx_encoder_params = dict(
    embeddings_noise=0.001,
    numeric_values=num_feature_params,
    embeddings=cat_feature_params)

# model_config['rnn_config']['additional_params']['theta'] = 100
trx_encoder = TrxEncoder(**trx_encoder_params)
if model_config['rnn_config']['additional_params']:
    rnn_encoder = NewRnnEncoder(
        input_size=trx_encoder.output_size,
        hidden_size=model_config['rnn_config']['hidden_state'],
        type=model_config['rnn_config']['rnn_type'],
        bidir=model_config['rnn_config']['bidir'],
        num_layers=model_config['rnn_config']['num_layers'],
        **model_config['rnn_config']['additional_params'])
else:
    rnn_encoder = NewRnnEncoder(
        input_size=trx_encoder.output_size,
        hidden_size=model_config['rnn_config']['hidden_state'],
        type=model_config['rnn_config']['rnn_type'],
        bidir=model_config['rnn_config']['bidir'],
        num_layers=model_config['rnn_config']['num_layers'])

seq_encoder = SimpleSeqEncoder(
    trx_encoder=trx_encoder,
    seq_encoder=rnn_encoder)

# coles model
coles_model = CoLESModule(
    seq_encoder=seq_encoder,
    optimizer_partial=partial(torch.optim.NAdam),
    lr_scheduler_partial=partial(torch.optim.lr_scheduler.ReduceLROnPlateau, mode='min', factor=0.2, patience=2)
)
# partial(torch.optim.lr_scheduler.StepLR, step_size=20, gamma=0.9) -
# ошибка MisconfigurationException: The provided lr scheduler `StepLR` doesn't follow PyTorch's LRScheduler API. You should override the `LightningModule.lr_scheduler_step` hook with your own logic if you are using a custom LR scheduler.

train_dl = PtlsDataModule(
    train_data = ColesDataset(MemoryMapDataset(data=df_data_train,i_filters=i_filters,),splitter=splitter),
    valid_data = ColesDataset(MemoryMapDataset(data=df_data_valid,i_filters=i_filters,),splitter=splitter),
    train_num_workers=model_config['num_workers'],
    train_batch_size=256#model_config['batch_size'],
)

## Обучение модели

In [ ]:
# pat = 5 if model_config['rnn_config']['rnn_type'] not in ['lem','lmu','lru','urnn','cornn'] else 3
early_stop_callback = EarlyStopping(monitor='recall_top_k', min_delta=0.0005, patience=5, verbose=False, mode='max')
trainer = pl.Trainer(
    max_epochs=5,
    gpus= 1 if torch.cuda.is_available() else 0,
    enable_progress_bar=True,
    callbacks = [early_stop_callback]
)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True, used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
%%time
gc.collect()
torch.cuda.empty_cache()
trainer.fit(coles_model, train_dataloaders=train_dl)

INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name               | Type             | Params
--------------------------------------------------------
0 | _loss              | ContrastiveLoss  | 0     
1 | _seq_encoder       | SimpleSeqEncoder | 17.0 K
2 | _validation_metric | BatchRecallTopK  | 0     
3 | _head              | Head             | 0     
--------------------------------------------------------
17.0 K    Trainable params
0         Non-trainable params
17.0 K    Total params
0.068     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

CPU times: user 9min 4s, sys: 1min 18s, total: 10min 23s
Wall time: 10min 36s


In [ ]:
# torch.save(coles_model.state_dict(), os.path.join(path_to_working_directory,'models',config_name.replace('yaml','pt')))

## Сохранение эмбеддингов

In [ ]:
def embedding_dataset(coles_model,prep_records_data,trainer,id_col,target_col=None):

  dl = inference_data_loader(prep_records_data, num_workers=1, batch_size=128)
  embeds = torch.vstack(trainer.predict(coles_model, dl))

  df = pd.DataFrame(data=embeds, columns=[f'embed_{i}' for i in range(embeds.shape[1])])
  df[id_col] = [x[id_col] for x in prep_records_data]
  if target_col:
    df[target_col] = [x[target_col] for x in prep_records_data]

  return df.drop_duplicates(subset=id_col)

In [ ]:
gc.collect()
torch.cuda.empty_cache()
train_embded_df = embedding_dataset(coles_model,df_data_train,trainer,data_config['id_col'],data_config['target_col'])
valid_embded_df = embedding_dataset(coles_model,df_data_valid,trainer,data_config['id_col'],data_config['target_col'])
# test_embded_df = embedding_dataset(coles_model,df_data_test,trainer,data_config['id_col'])

INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 179it [00:00, ?it/s]

INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 179it [00:00, ?it/s]

In [ ]:
train_embded_df.sample(3)

,embed_0,embed_1,embed_2,embed_3,embed_4,embed_5,embed_6,embed_7,embed_8,embed_9,...,embed_21,embed_22,embed_23,embed_24,embed_25,embed_26,embed_27,embed_28,user_id,target
2415,-0.989851,-1.363891,1.028471,1.082183,-4.968768,-5.347727,4.740883,1.732182,7.664949,0.883919,...,0.932838,-0.834365,1.647158,-1.428037,8.525152,-0.617094,-2.665794,3.249367,25797,0
51187,-2.296933,-0.849020,1.010534,0.589211,3.507089,0.358860,3.966241,1.722964,-3.425639,0.835751,...,0.128745,-1.310537,1.341649,-1.333171,8.030355,4.308943,-3.935693,2.236075,560533,1
48115,0.283400,-0.722170,1.266622,0.883271,1.315350,-0.623689,0.956419,0.392604,-0.440619,-0.086565,...,0.247292,0.232021,0.149486,0.310009,-0.781566,-1.703891,-2.302982,-0.313616,518024,0


In [ ]:
# train_embded_df.to_csv(os.path.join(data_config['path_folder'],'train_emb_'+config_name.replace('yaml','csv')),encoding='utf-8',index=False)
# valid_embded_df.to_csv(os.path.join(data_config['path_folder'],'valid_emb_'+config_name.replace('yaml','csv')),encoding='utf-8',index=False)
# test_embded_df.to_csv(os.path.join(data_config['path_folder'],'test_emb_'+config_name.replace('yaml','csv')),encoding='utf-8',index=False)

Оценим при помощи catboost наши эмбеддинги - код с примеров быстро вставил

In [ ]:
from catboost import CatBoostClassifier, metrics

In [ ]:
y_train = train_embded_df[data_config['target_col']].values
X_train = train_embded_df.drop([data_config['target_col'],data_config['id_col']], axis=1)
# if model_config['rnn_config']['rnn_type'] == 'indrnn':
X_train_emb = X_train
# else:
#     X_train_emb = pd.DataFrame(np.arange(len(X_train)))
#     X_train_emb['embeddings'] = X_train.values.tolist()
#     X_train_emb = X_train_emb.drop([0], axis=1)

y_val = valid_embded_df[data_config['target_col']].values
X_val = valid_embded_df.drop([data_config['target_col'],data_config['id_col']], axis=1)
# if model_config['rnn_config']['rnn_type'] == 'indrnn':
X_val_emb = X_val
# else:
#     X_val_emb = pd.DataFrame(np.arange(len(X_val)))
#     X_val_emb['embeddings'] = X_val.values.tolist()
#     X_val_emb = X_val_emb.drop([0], axis=1)

# if model_config['rnn_config']['rnn_type'] == 'indrnn':
CatBoostModel_emb = CatBoostClassifier(
  devices='gpu',
  iterations=1000,
  learning_rate=0.05,
  use_best_model=True,
  custom_metric=[metrics.Accuracy(),metrics.AUC()],
  random_seed=42,
  logging_level='Silent',
  # embedding_features=['embeddings'],
  depth=5
)
# else:
#     CatBoostModel_emb = CatBoostClassifier(
#         devices='gpu',
#         iterations=1000,
#         learning_rate=0.05,
#         use_best_model=True,
#         custom_metric=[metrics.Accuracy(),metrics.AUC()],
#         random_seed=42,
#         logging_level='Silent',
#         embedding_features=['embeddings'],
#         depth=5
#     )

CatBoostModel_emb.fit(
    X_train_emb, y_train,
    eval_set=(X_val_emb, y_val)
)

In [ ]:
from sklearn.metrics import roc_auc_score, accuracy_score,f1_score

val_preds = CatBoostModel_emb.predict(X_val_emb)
val_preds_proba = CatBoostModel_emb.predict_proba(X_val_emb)[:,1]
train_preds = CatBoostModel_emb.predict(X_train_emb)
train_preds_proba = CatBoostModel_emb.predict_proba(X_train_emb)[:,1]

results = [
            {'dataset':'train',
            'roc-auc':round(roc_auc_score(y_train,train_preds_proba,),3),
            'accuracy':round(accuracy_score(y_train,train_preds),3),
            'f1':round(f1_score(y_train,train_preds,average='micro'),3),
            'model_type' : model_config['rnn_config']['rnn_type'],
            'hidden_size' :  model_config['rnn_config']['hidden_state'],
            'num_layers' : model_config['rnn_config']['num_layers'],
            'bidir' : model_config['rnn_config']['bidir'],
            },
            {'dataset':'valid',
            'roc-auc':round(roc_auc_score(y_val,val_preds_proba),3),
            'accuracy':round(accuracy_score(y_val,val_preds),3),
            'f1':round(f1_score(y_val,val_preds,average='micro'),3),
            'model_type' : model_config['rnn_config']['rnn_type'],
            'hidden_size' :  model_config['rnn_config']['hidden_state'],
            'num_layers' : model_config['rnn_config']['num_layers'],
            'bidir' : model_config['rnn_config']['bidir'],
            }
           ]
pd.DataFrame(results)

,dataset,roc-auc,accuracy,f1,model_type,hidden_size,num_layers,bidir
0,train,0.749,0.917,0.917,s5,128,1,False
1,valid,0.689,0.916,0.916,s5,128,1,False


In [ ]:
pd.DataFrame(results).to_csv(os.path.join(path_to_working_directory,'results','metrics_'+config_name.replace('yaml','csv')),
                             encoding='utf-8',
                             index=False)